In [ ]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

import numpy as np
import resnet
print(resnet.__file__)
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import mlab
import matplotlib.cm as cm
import pickle
import LogHistory
import pandas as pd
import random
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,average_precision_score,confusion_matrix,recall_score,accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
import math
import gc
import sys
from string import Template

import tensorflow as tf

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet18.csv')

In [ ]:
output = open("D:/罗飞/kitchen_processed_data/"+'xall.p', 'rb')
xall=pd.read_pickle(output,compression=None)
output = open("D:/罗飞/kitchen_processed_data/"+'yall.p', 'rb')
yall=pd.read_pickle(output,compression=None)
output.close()

In [ ]:
le = preprocessing.LabelEncoder()
yall=le.fit_transform(yall)
num_classes=len(np.unique(yall))
print(np.unique(yall),num_classes)
xall, yall = shuffle(xall,yall)
print(yall[0:15])
x_train, x_test, y_train, y_test =train_test_split(xall, yall, test_size=0.2)

In [ ]:
print(xall.shape)

In [ ]:
# input image dimensions
img_rows, img_cols = 80, 80
# The CIFAR10 images are RGB.
img_channels = 2

In [ ]:
from keras_contrib.applications import resnet

In [ ]:
model = resnet.ResNet18(( img_rows, img_cols,img_channels), num_classes)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = LogHistory.LogHistory()
class_weight0 = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [ ]:
batch_size = 32
nb_epoch = 100
data_augmentation = True

In [ ]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=[lr_reducer, early_stopper, csv_logger])

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)

In [ ]:
from sklearn.metrics import f1_score,average_precision_score,confusion_matrix,recall_score,accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib import mlab
import matplotlib
import matplotlib.cm as cm
import itertools
font = {'family' : 'normal',
        'size'   : 22}

matplotlib.rc('font', **font)
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)
    plt.figure(figsize=(24,16))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=60,fontsize=17)
    plt.yticks(tick_marks, classes,fontsize=17)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
y_pred = model.predict(x_test,batch_size=200)
y_pred=np.argmax(y_pred, axis = 1)

print(accuracy_score(y_test,y_pred))
recall=recall_score(y_test, y_pred,average='macro')
print(recall)
print(f1_score(y_test,y_pred,average="macro"))

print(y_pred)
cnf_matrix=confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
np.set_printoptions(precision=1)
class_names=class_name=['Drinking',"Eating","Standing","Sitting","Walking","Open door and get in","Open door and get out","Washing",
           "Open oven","Close oven","Open cabinet","Close cabinet","Open fridge","Close fridge","No activity"]
# Plot non-normalized confusion matrix
#plot_confusion_matrix(cnf_matrix, classes=class_names,
#                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()